#### Buy decision
- if 10-period RSI of the stock is below 30
    - Buy on the next day's open
    
#### Sell decision
- if 10-period RSI is above 40 OR after 10 trading days
    - Sell on next days open

In [12]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [10]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = tickers['Symbol'].to_list()
tickers = [i.replace('.', '-') for i in tickers]

In [13]:
# Errors
tickers.pop(474)
tickers.pop(489)

'WU'

In [22]:
def RSIcalc(asset):
    df = yf.download(asset, start='2011-01-1')
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df['price change'] = df['Adj Close'].pct_change()
    df['Upmove'] = df['price change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['price change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['avg up'] = df['Upmove'].ewm(span=19).mean()
    df['avg down'] = df['Downmove'].ewm(span=19).mean()
    df = df.dropna()
    df['RS'] = df['avg up'] / df['avg down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100/(x+1)))
    df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy'] = 'Yes'
    df.loc[(df['Adj Close'] < df['MA200']) | (df['RSI'] > 30), 'Buy'] = 'No'
    return df

In [23]:
RSIcalc(tickers[0])

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,MA200,price change,Upmove,Downmove,avg up,avg down,RS,RSI,Buy
Date,,,,,,,,,,,,,,,
2011-10-14,78.940002,79.230003,78.120003,78.889999,60.409805,2840800,67.103449,0.014011,0.014011,0.000000,0.011144,0.006978,1.596943,61.493185,No
2011-10-17,78.629997,79.050003,75.660004,75.879997,58.104897,4359500,67.069641,-0.038155,0.000000,0.038155,0.010029,0.010096,0.993422,49.835013,No
2011-10-18,76.180000,79.150002,75.709999,78.360001,60.003960,4167200,67.043486,0.032683,0.032683,0.000000,0.012295,0.009086,1.353122,57.503273,No
2011-10-19,78.260002,78.860001,77.269997,77.699997,59.498554,3865000,67.015254,-0.008423,0.000000,0.008423,0.011065,0.009020,1.226767,55.091845,No
2011-10-20,77.769997,79.059998,77.110001,78.680000,60.248985,4087300,66.990775,0.012613,0.012613,0.000000,0.011220,0.008118,1.382134,58.020832,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-07,206.350006,206.809998,203.309998,203.729996,203.729996,1531100,176.329579,-0.011259,0.000000,0.011259,0.003488,0.002694,1.294785,56.422938,No
2021-06-08,202.000000,204.039993,201.139999,203.589996,203.589996,1701400,176.559298,-0.000687,0.000000,0.000687,0.003139,0.002493,1.259096,55.734501,No
2021-06-09,203.559998,203.570007,201.940002,202.740005,202.740005,1707600,176.783742,-0.004175,0.000000,0.004175,0.002825,0.002661,1.061565,51.493171,No
